In [2]:
from sentence_transformers import SentenceTransformer
from PIL import Image

import numpy as np
import openai
import pandas as pd
import pickle
import uuid
import pinecone
#from tqdm.auto import tqdm
import datetime
from time import sleep
import traceback
from dotenv import load_dotenv
import os
from matplotlib import pyplot as plt
from matplotlib import image as mpimg

In [4]:
load_dotenv()

IMAGE_PINECONE_API_KEY = os.getenv("IMAGE_PINECONE_API_KEY")
IMAGE_PINECONE_ENV = os.getenv("IMAGE_PINECONE_ENV")

In [6]:
# initialize connection (get API key at app.pinecone.io)
def initPinecone(index_name:str, pinecone_api_key:str, pinecone_env:str, dimension_len:int = 1536) -> pinecone.Index:
    pinecone.init(
        api_key=pinecone_api_key,
        #environment="asia-southeast1-gcp"  # find next to API key  #us-central1-gcp #asia-southeast1-gcp
        environment=pinecone_env
    )
        #environment="us-central1-gcp"  # find next to API key
    # check if index already exists (it shouldn't if this is first time)
    if index_name not in pinecone.list_indexes():
    # if does not exist, create index
        pinecone.create_index(
            index_name,
            dimension=dimension_len,
            metric='cosine',
            metadata_config={
                'indexed': ['title', 'heading']
            }
        )

    # connect to index
    return pinecone.Index(index_name)

In [7]:
def generate_vector(id, values, metadata):
    return {
        'id': id,
        'values': values,
        'metadata': { 'image_name': metadata }
    }

In [8]:
def seed(index: pinecone.Index):
    # Load CLIP model
    model = SentenceTransformer('clip-ViT-B-32')

    img_data = []

    img_data.append("001.png")
    img_data.append("002.png")
    img_data.append("003.png")
    img_data.append("004.png")

    to_upsert = []
    for item in img_data:
        uid = str(uuid.uuid4())
        embedding = model.encode(Image.open('./images/' + item))
        embedding_list = embedding.tolist()
        vector = generate_vector(id=uid,values=embedding_list,metadata=item)
        to_upsert.append(vector)
        #print(vector)
   
    # print(to_upsert)
    # view index stats
    """
    
    """
    print(index.describe_index_stats())

    index.upsert(vectors=to_upsert)

    # view index stats
    print(index.describe_index_stats())
    print(to_upsert)




In [63]:
def search(index: pinecone.Index, query:str):
    model = SentenceTransformer('clip-ViT-B-32')

    # Encode text query
    query_string = query
    text_emb = model.encode(query_string)

    print(text_emb)

    results = index.query(vector=text_emb,top_k=1)
    """
        result = results[0]
    print(result)
    plt.title(result)
    image = mpimg.imread('./images/' + result)
    plt.imshow(image)
    plt.show()
    """





In [9]:
index_name = "osha-images"

index = initPinecone(index_name=index_name,pinecone_api_key=IMAGE_PINECONE_API_KEY,pinecone_env=IMAGE_PINECONE_ENV,dimension_len=512)
seed(index)

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
[{'id': '1c0becf9-66c9-4991-91c1-ffe4ed230bed', 'values': [-0.3133637309074402, -0.3839135468006134, 0.1538304090499878, 0.034784168004989624, 0.4781484305858612, -0.2928144931793213, -0.19402647018432617, -0.29776567220687866, -0.040239375084638596, 0.10269118845462799, 0.08939486742019653, -0.0725654810667038, 1.0968003273010254, 0.2611043453216553, 0.5043379068374634, -0.17639967799186707, -0.45830854773521423, -0.12561142444610596, 0.03440430760383606, -0.43220698833465576, -0.03009471297264099, 0.46301329135894775, 0.27084776759147644, -0.027038410305976868, 0.014963477849960327, 0.22881974279880524, 0.0845421627163887, 0.3059176802635193, -0.17535793781280518, 0.08979449421167374, 0.1063983142375946, -0.020259976387023926, -0.2536364197731018, -0.027111515402793884, 0.625154972076416, -0.07743813842535019, 

In [64]:
index_name = "osha-images"

index = initPinecone(index_name=index_name,pinecone_api_key=IMAGE_PINECONE_API_KEY,pinecone_env=IMAGE_PINECONE_ENV,dimension_len=512)

search(index=index, query="open field")

[-5.87108694e-02 -1.53861865e-01  2.09312707e-01  2.11406693e-01
  1.51969284e-01  4.78777736e-02 -5.60188591e-02 -1.15776598e+00
  8.98227841e-02  4.88662899e-01 -2.84218639e-02 -2.44533435e-01
 -3.72980475e-01 -2.78696448e-01  4.27829862e-01 -4.72462326e-02
  1.90005869e-01  1.76985294e-01 -5.54078221e-02  8.87627378e-02
  8.87869895e-02  3.06002945e-01  2.99914539e-01 -1.83031887e-01
 -2.94887722e-01  3.90726104e-02 -1.29930243e-01  3.08872938e-01
 -1.51497573e-01 -2.26556823e-01 -2.23324075e-01 -1.31659329e-01
 -3.11659068e-01 -1.29265666e-01 -2.08148375e-01  2.93743730e-01
  1.46948099e-01  3.49780619e-02  4.68947142e-02  2.41101384e-02
 -7.40833730e-02 -2.58562446e-01  3.19743872e-01  1.81040391e-02
  3.14081371e-01 -8.69113207e-03 -1.11226730e-01 -1.72764957e-01
  2.27828920e-01  8.59902427e-02 -6.99984729e-02 -2.31381074e-01
  3.33244205e-01  1.53519183e-01  5.78380525e-02 -1.22549742e-01
 -9.35357809e-03  1.04955889e-01 -2.90258408e-01  2.89213836e-01
  1.21103227e-01 -1.87780

ApiValueError: Unable to prepare type ndarray for serialization